### **Tutorial 13: Building a Vanilla RNN with PyTorch for Sequential Data**
In this tutorial, we’ll learn how to build a vanilla Recurrent Neural Network (RNN) from scratch using PyTorch. A vanilla RNN is a type of neural network designed to process sequential data such as time series, text, or other ordered data. It has the ability to capture temporal dependencies by using its hidden state that gets updated as it processes each element in the sequence.

We will:

- Understand the basics of vanilla RNNs.
- Build a simple RNN model.
- Train the model on a sequential dataset.
- Evaluate the model.

**Prerequisites**
- Basic understanding of neural networks and RNNs.
- Install the following Python libraries: torch, numpy, matplotlib.

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import pandas as pd

In [2]:
class VanillaRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(VanillaRNN, self).__init__()
        self.hidden_size = hidden_size

        # Define layers: input + hidden --> hidden
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.tanh = nn.Tanh()

    def forward(self, x, hidden):
        combined = torch.cat((x, hidden), dim=1)
        hidden = self.tanh(self.i2h(combined))
        output = self.i2o(combined)
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)

In [3]:
import torch
import torch.nn as nn

class VanillaRNN2(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(VanillaRNN2, self).__init__()
        self.hidden_size = hidden_size

        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True) 
        self.i2o = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        out, hidden = self.rnn(x, hidden)
        last_out = out[:, -1, :]
        output = self.i2o(last_out)
        
        return output, hidden

    def init_hidden(self, batch_size):
        # Initialize hidden state with zeros
        return torch.zeros(1, batch_size, self.hidden_size)




In [4]:
def demo_simple_regression_data():

    data = {
        "x1": [2.0, 3.5, 1.8, 2.5, 3.0],
        "x2": [3000, 4000, 2800, 3500, 3700],
        "x3": [4, 6, 4, 6, 6],
        "y": [30, 20, 35, 25, 22]
    }

    df = pd.DataFrame(data)
    X = df[["x1", "x2", "x3"]].values
    y_actual = df['y'].values 
    X_normalized = (X - X.mean(axis=0)) / X.std(axis=0)

    inputs = torch.tensor(X_normalized, dtype=torch.float32).unsqueeze(1) #add unsqueeeze nn.RNN
    targets = torch.tensor(y_actual, dtype=torch.float32).view(-1, 1)

    input_size = 3  # 3 features
    hidden_size = 4  
    output_size = 1  
    num_epochs = 500  

    model = VanillaRNN2(input_size, hidden_size, output_size)
    criterion = nn.MSELoss()  
    optimizer = optim.SGD(model.parameters(), lr=0.01)


    print(f'input size: {inputs.size()}')
    print(f'output size: {targets.size()}')

    for epoch in range(num_epochs):
        model.train()  
        hidden = model.init_hidden(batch_size=inputs.size(0))     # 5 samples
        optimizer.zero_grad() 
        output, hidden = model(inputs, hidden)
        if epoch ==1:
            print(f"Hidden: {hidden.size()}")
        loss = criterion(output, targets)
        loss.backward()

        optimizer.step()
        if (epoch + 1) % 100 == 0:  
            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"Output: {output.view(-1).tolist()}")            
            print(f"Target: {targets.view(-1).tolist()}")
            print(f"Loss: {loss.item():.4f}\n")

demo_simple_regression_data()

input size: torch.Size([5, 1, 3])
output size: torch.Size([5, 1])
Hidden: torch.Size([1, 5, 4])
Epoch 100/500
Output: [31.85175323486328, 19.995031356811523, 31.908023834228516, 25.00372314453125, 22.097593307495117]
Target: [30.0, 20.0, 35.0, 25.0, 22.0]
Loss: 2.5998

Epoch 200/500
Output: [32.44231414794922, 19.97885513305664, 32.51970291137695, 24.953006744384766, 22.069580078125]
Target: [30.0, 20.0, 35.0, 25.0, 22.0]
Loss: 2.4249

Epoch 300/500
Output: [32.38294982910156, 19.97288703918457, 32.53993225097656, 24.91164779663086, 22.112144470214844]
Target: [30.0, 20.0, 35.0, 25.0, 22.0]
Loss: 2.3503

Epoch 400/500
Output: [31.615814208984375, 19.972278594970703, 32.6030158996582, 24.909557342529297, 22.093042373657227]
Target: [30.0, 20.0, 35.0, 25.0, 22.0]
Loss: 1.6748

Epoch 500/500
Output: [30.767078399658203, 20.020912170410156, 33.63182830810547, 24.96997833251953, 21.981700897216797]
Target: [30.0, 20.0, 35.0, 25.0, 22.0]
Loss: 0.4924

